In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str,
                         labels: np.ndarray) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.token_indexers import PretrainedBertIndexer

token_indexer = PretrainedBertIndexer(
    pretrained_model="bert-base-uncased",
    max_pieces=config.max_seq_len,
    do_lowercase=True,
 )
# apparently we need to truncate the sequence here, which is a stupid design decision
def tokenizer(s: str):
    return token_indexer.wordpiece_tokenizer(s)[:config.max_seq_len - 2]

02/03/2019 17:35:41 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/keitakurita/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:00, 671.05it/s]
251it [00:00, 894.90it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [[UNK],
  [UNK],
  the,
  edit,
  ##s,
  made,
  under,
  my,
  user,
  ##name,
  [UNK],
  [UNK],
  [UNK],
  were,
  reverted,
  ##?,
  [UNK],
  weren,
  ##',
  ##t,
  van,
  ##dal,
  ##isms,
  ##,,
  just,
  closure,
  on,
  some,
  [UNK],
  after,
  [UNK],
  voted,
  at,
  [UNK],
  [UNK],
  [UNK],
  [UNK],
  [UNK],
  please,
  don,
  ##',
  ##t,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  [UNK],
  retired,
  now,
  ##.,
  ##8,
  ##9,
  ##.,
  ##20,
  ##5,
  ##.,
  ##38,
  ##.,
  ##27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.wordpiece_indexer.PretrainedBertIndexer at 0x1a2b504550>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

We don't need to build the vocab: all that is handled by the token indexer

In [17]:
vocab = Vocabulary()

### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[  101,   100,  2005,  ...,     0,     0,     0],
          [  101,   100,   100,  ...,     0,     0,     0],
          [  101,  1045,  2069,  ...,     0,     0,     0],
          ...,
          [  101,   100,   100,  ...,     0,     0,     0],
          [  101,  1000,   100,  ...,     0,     0,     0],
          [  101,   100,  3521,  ...,  3109, 29612,   102]]),
  'tokens-offsets': tensor([[ 1,  2,  3,  ...,  0,  0,  0],
          [ 1,  2,  3,  ...,  0,  0,  0],
          [ 1,  2,  3,  ...,  0,  0,  0],
          ...,
          [ 1,  2,  3,  ...,  0,  0,  0],
          [ 1,  2,  3,  ...,  0,  0,  0],
          [ 1,  2,  3,  ..., 20, 21, 22]]),
  'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 1, 1]])},
 'id': ['00a54f5b8d29a2b7',
  '0023daf96917e0d0',
  '008faa76dd3eb890',
  

In [23]:
batch["tokens"]["tokens"]

tensor([[  101,   100,  2005,  ...,     0,     0,     0],
        [  101,   100,   100,  ...,     0,     0,     0],
        [  101,  1045,  2069,  ...,     0,     0,     0],
        ...,
        [  101,   100,   100,  ...,     0,     0,     0],
        [  101,  1000,   100,  ...,     0,     0,     0],
        [  101,   100,  3521,  ...,  3109, 29612,   102]])

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([64, 24])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder

bert_embedder = PretrainedBertEmbedder(
        pretrained_model="bert-base-uncased",
        top_layer_only=True, # conserve memory
)
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                            # we'll be ignoring masks so we'll need to set this to True
                                                           allow_unmatched_keys = True)

02/03/2019 17:35:44 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/keitakurita/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
02/03/2019 17:35:44 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/keitakurita/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/hy/1czs1y5j2d58zgkqx6w_wnpw0000gn/T/tmpwnhi0hjt
02/03/2019 17:35:47 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads"

In [28]:
BERT_DIM = word_embeddings.get_output_dim()

class BertSentencePooler(Seq2VecEncoder):
    def forward(self, embs: torch.tensor, 
                mask: torch.tensor=None) -> torch.tensor:
        # extract first token tensor
        return embs[:, 0]
    
    @overrides
    def get_output_dim(self) -> int:
        return BERT_DIM
    
encoder = BertSentencePooler(vocab)

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[  101,   100,  2005,  ...,     0,     0,     0],
         [  101,   100,   100,  ...,     0,     0,     0],
         [  101,  1045,  2069,  ...,     0,     0,     0],
         ...,
         [  101,   100,   100,  ...,     0,     0,     0],
         [  101,  1000,   100,  ...,     0,     0,     0],
         [  101,   100,  3521,  ...,  3109, 29612,   102]]),
 'tokens-offsets': tensor([[ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ...,  0,  0,  0],
         ...,
         [ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ..., 20, 21, 22]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]])}

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])

In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[ 5.5654e-01,  1.8664e-01,  3.6654e-01,  3.6568e-01,  3.5104e-01,
         -2.1512e-01],
        [ 4.2962e-01, -2.8918e-02,  3.6842e-01, -2.0817e-01,  2.6102e-01,
         -3.2780e-01],
        [ 3.8703e-01, -8.1013e-02, -4.3039e-02,  8.7068e-02, -3.4778e-02,
         -1.0288e-01],
        [ 5.1402e-01,  1.7842e-01,  5.9583e-02, -1.6438e-01,  8.5872e-03,
         -2.7806e-02],
        [ 5.5920e-01,  3.8523e-01,  1.9661e-01, -2.7572e-02,  1.0058e-01,
         -1.6167e-01],
        [ 5.2509e-01,  1.6955e-01,  6.7285e-02,  1.0717e-01,  2.7909e-01,
          3.8543e-02],
        [ 5.6904e-01,  2.4338e-01, -6.4044e-03, -6.4930e-02, -1.3761e-02,
         -2.2972e-01],
        [ 5.6019e-01,  8.8073e-02,  2.9777e-02, -1.0398e-01,  9.6796e-02,
         -2.6863e-01],
        [ 5.3734e-01,  3.4953e-01,  1.8585e-02,  8.1381e-02,  1.0827e-01,
         -1.2286e-01],
        [ 1.1801e-01,  7.4278e-02, -8.8033e-02, -1.0108e-01,  4.5776e-02,
         -1.5425e-01],
        [ 5.1139e-01, -8.1566e

In [36]:
model(**batch)

{'class_logits': tensor([[ 4.6100e-01,  1.7326e-01,  2.2254e-01,  1.1410e-01,  2.4407e-01,
          -1.5805e-01],
         [ 6.3738e-01,  2.2056e-01,  2.9089e-02, -2.3310e-02,  1.3608e-01,
          -2.5500e-01],
         [ 2.8956e-01,  5.2109e-02, -5.5303e-02, -1.6900e-01, -1.0136e-01,
           9.3308e-02],
         [ 1.7099e-01,  1.3054e-01, -9.5852e-02, -2.4803e-01, -4.0757e-02,
          -1.8549e-01],
         [ 7.5998e-01,  2.4915e-01,  3.0788e-01,  1.6619e-01,  2.8302e-01,
           1.4241e-02],
         [ 4.6951e-01,  1.4172e-01, -3.9608e-02, -2.2060e-01, -1.5100e-01,
           4.1976e-03],
         [ 4.6982e-01,  2.5360e-01, -3.1616e-02, -2.1575e-01,  2.8362e-02,
           1.2691e-01],
         [ 5.4504e-01,  1.3360e-01,  2.0855e-01, -9.0100e-02,  2.3306e-01,
          -3.2395e-01],
         [ 5.2233e-01,  2.6013e-01,  9.8824e-02,  1.0170e-01,  2.2584e-01,
          -7.9434e-04],
         [ 2.9987e-01,  6.7261e-02, -4.3380e-02, -1.8769e-01, -2.1650e-01,
           1.3009e

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.7290, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [39]:
loss.backward()

In [40]:
[x.grad for x in list(model.encoder.parameters())]

[]

# Train

In [41]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [42]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [43]:
metrics = trainer.train()

02/03/2019 17:35:58 - INFO - allennlp.training.trainer -   Beginning training.
02/03/2019 17:35:58 - INFO - allennlp.training.trainer -   Epoch 0/1
02/03/2019 17:35:58 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 1287.852032
02/03/2019 17:35:58 - INFO - allennlp.training.trainer -   Training
loss: 0.6990 ||: 100%|██████████| 5/5 [00:33<00:00,  6.84s/it]
02/03/2019 17:36:31 - INFO - allennlp.training.trainer -                     Training |  Validation
02/03/2019 17:36:31 - INFO - allennlp.training.trainer -   loss          |     0.699  |       N/A
02/03/2019 17:36:31 - INFO - allennlp.training.trainer -   cpu_memory_MB |  1287.852  |       N/A
02/03/2019 17:36:31 - INFO - allennlp.training.trainer -   Epoch duration: 00:00:33
02/03/2019 17:36:31 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:00:33
02/03/2019 17:36:31 - INFO - allennlp.training.trainer -   Epoch 1/1
02/03/2019 17:36:31 - INFO - allennlp.training.trainer -   Peak CPU memor